In [1]:
#| include: false

import duckdb
import pandas as pd
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

%sql CREATE OR REPLACE TABLE user_actions AS SELECT * FROM read_csv('00_data/sql/user_actions.csv', header=True, columns={'user_id': 'INT', 'order_id': 'INT', 'action': 'VARCHAR', 'time': 'TIMESTAMP'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE courier_actions AS SELECT * FROM read_csv('00_data/sql/courier_actions.csv', header=True, columns={'courier_id': 'INT', 'order_id': 'INT', 'action': 'VARCHAR', 'time': 'TIMESTAMP'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE orders AS SELECT * FROM read_csv('00_data/sql/orders.csv', header=True, columns={'order_id': 'INT', 'creation_time': 'TIMESTAMP', 'product_ids': 'INT[]'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE users AS SELECT * FROM read_csv('00_data/sql/users.csv', header=True, columns={'user_id': 'INT', 'birth_date': 'DATE', 'sex': 'VARCHAR'}, dateformat='%d/%m/%y');
%sql CREATE OR REPLACE TABLE couriers AS SELECT * FROM read_csv('00_data/sql/couriers.csv', header=True, columns={'courier_id': 'INT', 'birth_date': 'DATE', 'sex': 'VARCHAR'}, dateformat='%d/%m/%y');
%sql CREATE OR REPLACE TABLE products AS SELECT * FROM read_csv('00_data/sql/products.csv', header=True, columns={'product_id': 'INT', 'name': 'VARCHAR', 'price': 'DOUBLE'});

,Count
0,87


# Аналіз продуктових метрик

Минулого разу ми навчилися будувати повноцінні дашборди і представляти результати аналізу у зрозумілій бізнесі формі — це дуже важлива навичка, необхідна будь-якому аналітику.

Цього разу ми трохи докладніше поговоримо про аналіз найважливіших метрик, які дозволяють з різних боків оцінити те, наскільки добре працює наш продукт.

Насправді раніше ми вже рахували окремі показники — наприклад, метрики залучення користувачів **DAU**, **WAU** та **MAU**, кількість замовлень, частку скасованих замовлень тощо. Це дійсно важливі показники, але часто представники бізнесу в першу чергу звертають увагу на більш зрозумілі показники, виражені в грошовій формі. Це можуть бути виручка, витрати чи рентабельність, а також відносні показники на кшталт доходу на одного користувача та середнього чека – все це теж важливо вміти розраховувати, чим ми з вами й займемося.

Крім того, наприкінці ми обговоримо методику розрахунку ще одного важливого показника – **Retention rate**.

На цей раз вам знову буде запропоновано написати кілька SQL-запитів і візуалізувати їх результат за допомогою графіків.

## Задача 1

Почнемо з виручки - найбільш загального показника, який покаже, який дохід приносить наш сервіс.

::::: {.callout-note icon=false}
## Завдання
:::: {#exr-sql-metrics-01}
<br>
Для кожного дня в таблиці `orders` розрахуйте такі показники:

1. Виручку, одержану в цей день.
2. Сумарний виторг на поточний день.
3. Приріст виручки, отриманої цього дня, щодо значення виручки за попередній день.

Колонки з показниками назвіть відповідно `revenue`, `total_revenue`, `revenue_change`. Колонку з датами назвіть `date`.

Приріст виручки розрахуйте у відсотках та округліть значення **до двох знаків після коми**.

Результат має бути відсортований за зростанням дати.

Поля в результуючій таблиці: `date`, `revenue`, `total_revenue`, `revenue_change`

**Пояснення:**

Вважатимемо, що оплата за замовлення надходить відразу після його оформлення, тобто випадки, коли замовлення було оформлено в один день, а оплата отримана наступного, виникнути не можуть.

Сумарна виручка на поточний день – це результат складання виручки, отриманої поточного дня, зі значеннями аналогічного показника всіх попередніх днів.

При розрахунку виручки пам'ятайте, що не всі замовлення були сплачені, деякі були скасовані користувачами.

Не забувайте при діленні заздалегідь переводити значення до потрібного типу даних. Пропущені значення приросту для першої дати не заповнюйте - просто залиште поля в цьому рядку порожніми.

::: {.callout-note collapse="true"}
## Підказка на випадок, якщо зовсім не виходить

Для вирішення завдання вам знадобиться інформація про замовлення з таблиці `orders` та ціни на товари з таблиці `products`. Щоб порахувати виторг для кожного дня, спочатку необхідно порахувати вартість кожного замовлення. Це можна зробити, склавши ціни товарів, що входять на замовлення. Щоб правильно приєднати дані про ціни на товари, списки із вмістом замовлень потрібно попередньо розширити за допомогою функції `unnest`. Після того як для кожного дня буде порахована сумарна вартість усіх замовлень (виручка), за допомогою віконних функцій можна порахувати суму наростаючим підсумком (загальну виручку) та приріст виручки (різницю між виручкою в поточний день та виручкою в попередній день, поділену на виручку в попередній день).
:::


::::
:::::